# Covid Linear Modeling
Rachel Epperson 1/21/2021

## Import Needed Packages 

In [ ]:
## Import Needed Packages -- Linear Regression ##
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from matplotlib.pyplot import figure

## Import Needed Packages -- Autoregression ##
from statsmodels.tsa.ar_model import AutoReg
import statsmodels.api as sm

## Import Needed Packages -- ARMA ##
from statsmodels.tsa.arima.model import ARIMA
from pandas import DataFrame

## Import Needed Packages -- ARIMA ##
from sklearn.metrics import mean_squared_error
from math import sqrt

### Name & Call Excel Sheet 

In [ ]:
## Dataframe with COVID data ##
covid_excel_info = pd.read_csv('us_states_covid19_daily.csv')
covid_excel_info.head()

### Separate Georgia from Rest of Data

In [ ]:
## Separating Georgia Info from Other States ##
Georgia_ind = covid_excel_info['state'] == 'GA'#find all rows with Georgia Info
Georgia_df = covid_excel_info[Georgia_ind] #pull out and separate

### Convert the Dates in the Dataframe to "Datetime" Objects

In [ ]:
## Cases: Dates as string (from int) ##
Georgia_date = Georgia_df['date'].astype(str) #change interger dates to strings
Georgia_date = [ datetime(year=int(i[0:4]), month=int(i[4:6]), day=int(i[6:8])) for i in Georgia_date ]  #make a datetime object
Georgia_df['date'] = Georgia_date #writing over original 'date' to updated dates

### Removing Empting Values in Excel Columns

In [ ]:
## Remove Empty Cells ##
Georgia_df['positive'].dropna() #delete all rows w/ 'positive' column that read no data "NaN"

### Calculate Delta Days from Datetime Objects ##

In [ ]:
## Convert Date Objects to deltaDays ## 
dates_series = list(Georgia_df['date']) #make lsit
Georgia_df['deltaDays'] = [ (d - dates_series[-1]).days for d in dates_series ] #iterate through to calculate delta days
Georgia_df = Georgia_df.sort_values(by = ['deltaDays']) #sort deltaDays starting at 0

### Plotting actual COVID cases in GA

In [ ]:
## Creating X and y ##
X = Georgia_df['deltaDays'].values.reshape(-1,1) #change in days #fitting to model
y = Georgia_df['positive'] #number of positives in each state (also column)
print(len(X))

## Ploting X vs. y for Actual Data Line -- Divided by 1000 for Aestheics##
figure(figsize=(10, 5), dpi=80)
plt.plot(X[0:222],y[0:222]/1000, color =  'blue', label = 'Training Data Set')
plt.plot(X[222:len(X)],y[222:len(X)]/1000, color =  'green', label = 'Testing Data Set')
plt.axvline(222, color = 'black')

## Title Labeling ##
plt.title("Real COVID-19 Cases in Georgia", fontsize=20) 

## Axis Labeling ##
plt.xlabel("Days", fontsize = 16) 
plt.ylabel("#Positive (x1000)", fontsize = 16) 
plt.legend(fontsize = 16)
plt.show()

## Linear Regression


In [ ]:
X = Georgia_df['deltaDays'].values.reshape(-1,1)
# all positive cases #
positiveCases = Georgia_df['positive'].values

### TrainTest Split

In [ ]:
size = int(len(y) * 0.8) #80% of data
y_train, y_test = positiveCases[0:size], positiveCases[size:len(y)]

X_train, X_test = X[0:size], X[size:len(y)]

### Train Model

In [ ]:
model_fit = LinearRegression().fit(X_train, y_train)

### Evaluate Model

In [ ]:
y_test_predict = model_fit.predict(X_test)

### Plot

In [ ]:
# Quilt rows and columns ##
nCols=2
nRows=1
## Spacers ##
wspace= 0.5
hspace= 0
## Quilt width and height ##
figsize_x = 10+(wspace*nCols)
figsize_y = 5*(nRows/nCols)+(hspace*nRows)

## Plot Grid ##
fig, ax = plt.subplots(nrows=nRows, ncols=nCols, figsize=(figsize_x, figsize_y),
gridspec_kw = {'wspace':wspace, 'hspace':hspace})

## Testing Line: #Real vs. Predicted ##
ax[0].plot(X_test, y_test/1000, color = 'green');
## Real Data Line##
ax[0].plot(X[size:len(X)], positiveCases[size:len(X)]/1000, color = 'blue', alpha = 0.5)
## Title ##
ax[0].set_title('%s'% ('ULR Forecast Line Plot'), fontsize=20, color='black');
## Axes ##
ax[0].set_xlabel('%s'% ('Days'), fontsize=16, color='black');
ax[0].set_ylabel('%s'% ('#Positive (x1000)'), fontsize=16, color='black');
ax[0].legend(['Predicted', 'Actual'])

## Residual: #Positive vs. Date ##
residuals = y_test_predict - y_test
ax[1].plot(X[size:len(X)], residuals/1000, color = 'red' );
## Title ##
ax[1].set_title('%s'% ('ULR Fit Residual Error Line Plot'), fontsize=20, color='black');
## Axes ##
ax[1].set_xlabel('%s'% ('Days'), fontsize=16, color='black');
ax[1].set_ylabel('%s'% ('#Positive (x1000)'), fontsize=16, color='black');
ax[1].legend(['adj. R² = 0.9566'])

### Plot only Residual

In [ ]:
figure(figsize=(10, 5), dpi=80)
plt.plot(X[size:len(X)], residuals/1000, color = 'red' )
plt.title('%s'% ('ULR Fit Residual Error Line Plot'), fontsize=20, color='black')
plt.xlabel('%s'% ('Days'), fontsize=16, color='black')
plt.ylabel('%s'% ('#Positive (x1000)'), fontsize=16, color='black')
plt.legend(['adj. R² = 0.9566'], prop={'size': 16})

### Adjusted R2 Value LR

In [ ]:
1 - (1-model_fit.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1)

### Equation Substitutions

In [ ]:
print(model_fit.coef_)
print(model_fit.intercept_)

## Autoregession

In [ ]:
X = Georgia_df['deltaDays'].values.reshape(-1,1)
# all positive cases #
positiveCases = Georgia_df['positive'].values

### TrainTest Split

In [ ]:
size = int(len(y) * 0.8) #80% of data
y_train, y_test = positiveCases[0:size], positiveCases[size:len(y)]

X_train, X_test = X[0:size], X[size:len(y)]

### Train Model

In [ ]:
model = AutoReg(y_test, lags = 1)
model_fit_AR = model.fit()

### Evalute Model

In [ ]:
y_test_predict_AR = model_fit_AR.predict()

### Plot

In [ ]:
# Quilt rows and columns ##
nCols=2
nRows=1
## Spacers ##
wspace= 0.5
hspace= 0
## Quilt width and height ##
figsize_x = 10+(wspace*nCols)
figsize_y = 5*(nRows/nCols)+(hspace*nRows)

## Plot Grid ##
fig, ax = plt.subplots(nrows=nRows, ncols=nCols, figsize=(figsize_x, figsize_y),
gridspec_kw = {'wspace':wspace, 'hspace':hspace})

## Testing Line: #Real vs. Predicted ##
ax[0].plot(X_test, y_test/1000, color = 'green');
## Real Data Line##
ax[0].plot(X[size:len(X)], positiveCases[size:len(X)]/1000, color = 'blue', alpha = 0.5)
## Title ##
ax[0].set_title('%s'% ('AR Forecast Line Plot'), fontsize=20, color='black');
## Axes ##
ax[0].set_xlabel('%s'% ('Days'), fontsize=16, color='black');
ax[0].set_ylabel('%s'% ('#Positive (x1000)'), fontsize=16, color='black');
ax[0].legend(['Predicted', 'Actual'])

## Residual: #Positive vs. Date ##
residuals = y_test_predict_AR - y_test[1:]
ax[1].plot(X[size + 1:len(X)], residuals/1000, color = 'red' );
## Title ##
ax[1].set_title('%s'% ('AR Fit Residual Error Line Plot'), fontsize=20, color='black');
## Axes ##
ax[1].set_xlabel('%s'% ('Days'), fontsize=16, color='black');
ax[1].set_ylabel('%s'% ('#Positive (x1000)'), fontsize=16, color='black');
ax[1].legend(['adj. R² = 0.9995'])

### Plot only Residuals

In [ ]:
figure(figsize=(10, 5), dpi=80)
residuals = y_test_predict_AR - y_test[1:]
plt.plot(X[size + 1:len(X)], residuals/1000, color = 'red')
plt.title('%s'% ('AR Fit Residual Error Line Plot'), fontsize=20, color='black')
plt.xlabel('%s'% ('Days'), fontsize=16, color='black')
plt.ylabel('%s'% ('#Positive (x1000)'), fontsize=16, color='black')
plt.legend(['adj. R² = 0.9995'], prop={'size': 16})

### Equation Substitutions AR

In [ ]:
print(model_fit_AR.summary())

### Adjusted R2 Value AR

In [ ]:
model = sm.OLS(y_test_predict_AR, X[size+1:len(X)]).fit()
print(model.rsquared_adj)

## ARMA

In [ ]:
X = Georgia_df['deltaDays'].values.reshape(-1,1)
# all positive cases #
positiveCases = Georgia_df['positive'].values

### TrainTest Split

In [ ]:
size = int(len(y) * 0.8) #80% of data
y_train, y_test = positiveCases[0:size], positiveCases[size:len(y)]

X_train, X_test = X[0:size], X[size:len(y)]

### Train Data

In [ ]:
model = ARIMA(y_test, order = (1, 0, 1))
model_fit_ARMA = model.fit()

### Evaluate Data

In [ ]:
y_test_predict_ARMA = model_fit_ARMA.predict()

### Plot

In [ ]:
# Quilt rows and columns ##
nCols=2
nRows=1
## Spacers ##
wspace= 0.5
hspace= 0
## Quilt width and height ##
figsize_x = 10+(wspace*nCols)
figsize_y = 5*(nRows/nCols)+(hspace*nRows)

## Plot Grid ##
fig, ax = plt.subplots(nrows=nRows, ncols=nCols, figsize=(figsize_x, figsize_y),
gridspec_kw = {'wspace':wspace, 'hspace':hspace})

## Testing Line: #Real vs. Predicted ##
ax[0].plot(X_test, y_test/1000, color = 'green');
## Real Data Line##
ax[0].plot(X[size:len(X)], positiveCases[size:len(X)]/1000, color = 'blue', alpha = 0.5)
## Title ##
ax[0].set_title('%s'% ('ARMA Forecast Line Plot'), fontsize=20, color='black');
## Axes ##
ax[0].set_xlabel('%s'% ('Days'), fontsize=16, color='black');
ax[0].set_ylabel('%s'% ('#Positive (x1000)'), fontsize=16, color='black');
ax[0].legend(['Predicted', 'Actual'])

## Residual: #Positive vs. Date ##
residuals = y_test_predict_ARMA - y_test
ax[1].plot(X[size + 1:len(X)], residuals[1:]/1000, color = 'red' );
## Title ##
ax[1].set_title('%s'% ('ARMA Fit Residual Error Line Plot'), fontsize=20, color='black');
## Axes ##
ax[1].set_xlabel('%s'% ('Days'), fontsize=16, color='black');
ax[1].set_ylabel('%s'% ('#Positive (x1000)'), fontsize=16, color='black');
ax[1].legend(['adj. R² = 0.9993'])

### Plot only Residuals

In [ ]:
figure(figsize=(10, 5), dpi=80)
residuals = y_test_predict_ARMA - y_test
plt.plot(X[size + 1:len(X)], residuals[1:]/1000, color = 'red')
plt.title('%s'% ('ARMA Fit Residual Error Line Plot'), fontsize=20, color='black')
plt.xlabel('%s'% ('Days'), fontsize=16, color='black')
plt.ylabel('%s'% ('#Positive (x1000)'), fontsize=16, color='black')
plt.legend(['adj. R² = 0.9993'], prop={'size': 16})

### Equation Substitutions ARMA

In [ ]:
print(model_fit_ARMA.summary())

### Adjusted R2 Value ARMA

In [ ]:
model = sm.OLS(y_test_predict_ARMA, X[size:len(X)]).fit()
print(model.rsquared_adj) #adjusted R^2 val

## ARIMA

In [ ]:
X = Georgia_df['deltaDays'].values.reshape(-1,1)
# all positive cases #
positiveCases = Georgia_df['positive'].values

### TrainTest Split

In [ ]:
size = int(len(y) * 0.8) #80% of data
y_train, y_test = positiveCases[0:size], positiveCases[size:len(y)]

X_train, X_test = X[0:size], X[size:len(y)]

### Train Data

In [ ]:
model = ARIMA(y_test, order = (1, 1, 1))
model_fit_ARIMA = model.fit()

### Evaluate Data

In [ ]:
y_test_predict_ARIMA = model_fit_ARIMA.predict()

### Plot

In [ ]:
# Quilt rows and columns ##
nCols=2
nRows=1
## Spacers ##
wspace= 0.5
hspace= 0
## Quilt width and height ##
figsize_x = 10+(wspace*nCols)
figsize_y = 5*(nRows/nCols)+(hspace*nRows)

## Plot Grid ##
fig, ax = plt.subplots(nrows=nRows, ncols=nCols, figsize=(figsize_x, figsize_y),
gridspec_kw = {'wspace':wspace, 'hspace':hspace})

## Testing Line: #Real vs. Predicted ##
ax[0].plot(X_test, y_test/1000, color = 'green');
## Real Data Line##
ax[0].plot(X[size:len(X)], positiveCases[size:len(X)]/1000, color = 'blue', alpha = 0.5)
## Title ##
ax[0].set_title('%s'% ('ARIMA Forecast Line Plot'), fontsize=20, color='black');
## Axes ##
ax[0].set_xlabel('%s'% ('Days'), fontsize=16, color='black');
ax[0].set_ylabel('%s'% ('#Positive (x1000)'), fontsize=16, color='black');
ax[0].legend(['Predicted', 'Actual'])

## Residual: #Positive vs. Date ##
residuals = y_test_predict_ARIMA - y_test
ax[1].plot(X[size + 1:len(X)], residuals[1:]/1000, color = 'red' );
## Title ##
ax[1].set_title('%s'% ('ARIMA Fit Residual Error Line Plot'), fontsize=20, color='black');
## Axes ##
ax[1].set_xlabel('%s'% ('Days'), fontsize=16, color='black');
ax[1].set_ylabel('%s'% ('#Positive (x1000)'), fontsize=16, color='black');
ax[1].legend(['adj. R² = 0.9852'])

### Plot only Residuals

In [ ]:
figure(figsize=(10, 5), dpi=80)
residuals = y_test_predict_ARIMA - y_test
plt.plot(X[size + 1:len(X)], residuals[1:]/1000, color = 'red')
plt.title('%s'% ('ARIMA Fit Residual Error Line Plot'), fontsize=20, color='black')
plt.xlabel('%s'% ('Days'), fontsize=16, color='black')
plt.ylabel('%s'% ('#Positive (x1000)'), fontsize=16, color='black')
plt.legend(['adj. R² = 0.9852'], prop={'size': 16})

### Equation Substitutions ARIMA

In [ ]:
print(model_fit_ARIMA.summary())

### Adjusted R2 Value ARIMA

In [ ]:
model = sm.OLS(y_test_predict_ARIMA, X[size:len(X)]).fit()
print(model.rsquared_adj) #adjusted R^2 val